In [1]:
# !sudo apt update
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
# !tar xf spark-3.2.1-bin-hadoop3.2.tgz
# !pip install -q findspark
# !pip install pyspark

In [2]:
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


In [1]:
# import findspark
# findspark.init()
# findspark.find()
import pyspark
from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F


In [2]:
spark = SparkSession \
       .builder \
       .appName("House Price Prediction") \
       .getOrCreate()

spark

Read Data


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
file_location1 = "train.csv"
file_location2 = "test.csv"
file_type = "csv"

# CSV options
infer_schema = True
first_row_is_header = True
delimiter = ","

train_df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location1)

test_df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location2)

In [5]:
type(train_df)

pyspark.sql.dataframe.DataFrame

In [6]:
train_df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |--

In [7]:
test_df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |--

In [8]:
train_df.show(5, False)

+---+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+---------+
|Id |MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition

In [9]:
type(train_df)

pyspark.sql.dataframe.DataFrame

In [10]:
housetrain_df = train_df.toPandas()
housetrain_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65,8450,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,2,2008,WD,Normal,208500
1,2,20,RL,80,9600,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,5,2007,WD,Normal,181500
2,3,60,RL,68,11250,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,9,2008,WD,Normal,223500
3,4,70,RL,60,9550,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84,14260,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,12,2008,WD,Normal,250000


In [11]:
train_df.describe().toPandas()

,summary,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,count,1460,1460,1460,1460,1460,1460,1460,1460,1460,...,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460
1,mean,730.5,56.897260273972606,None,70.04995836802665,10516.828082191782,None,None,None,None,...,2.758904109589041,None,None,None,43.489041095890414,6.321917808219178,2007.8157534246575,None,None,180921.19589041095
2,stddev,421.6100093688479,42.30057099381045,None,24.28475177448321,9981.26493237915,None,None,None,None,...,40.17730694453021,None,None,None,496.1230244579441,2.7036262083595113,1.3280951205521145,None,None,79442.50288288663
3,min,1,20,C (all),100,1300,Grvl,Grvl,IR1,Bnk,...,0,Ex,GdPrv,Gar2,0,1,2006,COD,Abnorml,34900
4,max,1460,190,RM,NA,215245,Pave,Pave,Reg,Lvl,...,738,NA,NA,TenC,15500,12,2010,WD,Partial,755000


Check Missing Values

In [12]:
from pyspark.sql.functions import isnull, when, count, col
train_df.select([count(when(isnull(c), c)).alias(c) for c in train_df.columns]).show()

+---+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+---------+
| Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition

In [13]:
from pyspark.sql.functions import col,isnan,when,count


In [14]:
df2 = train_df.select([count(when((col(c)=='NA') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in train_df.columns])


In [15]:
df2.show()

+---+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+---------+
| Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition

In [16]:
df3 = train_df.drop('Alley','PoolQC','Fence','MiscFeature','FireplaceQu')

In [17]:
df3.show()

+---+----------+--------+-----------+-------+------+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+-------+------+------+--------+-------------+---------+
| Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition2|BldgType|HouseStyle|OverallQual|OverallCond|Yea

In [18]:
df4 = df3.select([count(when((col(c)=='NA') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df3.columns])

In [19]:
df4.show()

+---+----------+--------+-----------+-------+------+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+-------+------+------+--------+-------------+---------+
| Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition2|BldgType|HouseStyle|OverallQual|OverallCond|Yea

In [20]:
df3.describe()

DataFrame[summary: string, Id: string, MSSubClass: string, MSZoning: string, LotFrontage: string, LotArea: string, Street: string, LotShape: string, LandContour: string, Utilities: string, LotConfig: string, LandSlope: string, Neighborhood: string, Condition1: string, Condition2: string, BldgType: string, HouseStyle: string, OverallQual: string, OverallCond: string, YearBuilt: string, YearRemodAdd: string, RoofStyle: string, RoofMatl: string, Exterior1st: string, Exterior2nd: string, MasVnrType: string, MasVnrArea: string, ExterQual: string, ExterCond: string, Foundation: string, BsmtQual: string, BsmtCond: string, BsmtExposure: string, BsmtFinType1: string, BsmtFinSF1: string, BsmtFinType2: string, BsmtFinSF2: string, BsmtUnfSF: string, TotalBsmtSF: string, Heating: string, HeatingQC: string, CentralAir: string, Electrical: string, 1stFlrSF: string, 2ndFlrSF: string, LowQualFinSF: string, GrLivArea: string, BsmtFullBath: string, BsmtHalfBath: string, FullBath: string, HalfBath: string

In [21]:
df3.printSchema()


root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |-- Exterior2nd: string (nullable = true

In [22]:
df3.toPandas().isnull().sum()


Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
Length: 76, dtype: int64

In [23]:
#from pyspark.sql.functions import when, lit
#df5 = df3.withColumn('GarageFinish', when(df3['GarageArea']==0,
#lit(0)).otherwise(df3['GarageFinish']))

In [24]:
df5 = df3.drop('Id')

In [25]:
df5.printSchema()

root
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |-- Exterior2nd: string (nullable = true)
 |-- MasVnrType: string (nullable

In [26]:
train_string_columns = []

for col, dtype in df5.dtypes:
    if dtype == 'string':
        train_string_columns.append(col)


In [27]:
train_string_columns

['MSZoning',
 'LotFrontage',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [28]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

indexers = [StringIndexer(inputCol=column, outputCol=column+'_index', handleInvalid='keep').fit(df5) for column in train_string_columns]


pipeline = Pipeline(stages=indexers)
train_indexed = pipeline.fit(df5).transform(df5)


In [29]:
train_indexed.printSchema()

root
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |-- Exterior2nd: string (nullable = true)
 |-- MasVnrType: string (nullable

In [30]:
train_indexed.show()

+----------+--------+-----------+-------+------+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+-------+------+------+--------+-------------+---------+--------------+-----------------+------------+--------------+-----------------+---------------+---------------+---------------+------------------+----------------+----------------+----------

In [31]:
df6= train_indexed.drop(*train_string_columns)

In [32]:
df6.printSchema()

root
 |-- MSSubClass: integer (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- BsmtFinSF1: integer (nullable = true)
 |-- BsmtFinSF2: integer (nullable = true)
 |-- BsmtUnfSF: integer (nullable = true)
 |-- TotalBsmtSF: integer (nullable = true)
 |-- 1stFlrSF: integer (nullable = true)
 |-- 2ndFlrSF: integer (nullable = true)
 |-- LowQualFinSF: integer (nullable = true)
 |-- GrLivArea: integer (nullable = true)
 |-- BsmtFullBath: integer (nullable = true)
 |-- BsmtHalfBath: integer (nullable = true)
 |-- FullBath: integer (nullable = true)
 |-- HalfBath: integer (nullable = true)
 |-- BedroomAbvGr: integer (nullable = true)
 |-- KitchenAbvGr: integer (nullable = true)
 |-- TotRmsAbvGrd: integer (nullable = true)
 |-- Fireplaces: integer (nullable = true)
 |-- GarageCars: integer (nullable = true)
 |-

In [33]:
df6.describe().toPandas()

,summary,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,KitchenQual_index,Functional_index,GarageType_index,GarageYrBlt_index,GarageFinish_index,GarageQual_index,GarageCond_index,PavedDrive_index,SaleType_index,SaleCondition_index
0,count,1460,1460,1460,1460,1460,1460,1460,1460,1460,...,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460
1,mean,56.897260273972606,10516.828082191782,6.0993150684931505,5.575342465753424,1971.267808219178,1984.8657534246574,443.6397260273973,46.54931506849315,567.2404109589041,...,0.6184931506849315,0.15616438356164383,0.6595890410958904,25.596575342465755,0.9376712328767123,0.1684931506849315,0.14794520547945206,0.10273972602739725,0.23356164383561645,0.3116438356164384
2,stddev,42.30057099381045,9981.26493237915,1.3829965467415926,1.1127993367127318,30.202904042525294,20.64540680770938,456.0980908409278,161.3192728065416,441.8669552924343,...,0.7304691071299438,0.6635494109714145,1.0399213582806461,22.617136758938415,0.93493908426687,0.5739075714519857,0.5441720862963024,0.3652006509951003,0.793805472262832,0.7673968418056222
3,min,20,1300,1,1,1872,1950,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,max,190,215245,10,9,2010,2010,5644,1474,2336,...,3.0,6.0,6.0,97.0,3.0,5.0,5.0,2.0,8.0,5.0


In [34]:
# print("Dropping outliers resulted in %d instances in the new dataset" % len(df6))
# pd_train = df6.drop(pd_train[(pd_train['GrLivArea']>4500) 
#                                 & (pd_train['SalePrice']<300000)].index)
# print("Dropping outliers resulted in %d instances in the new dataset" % len(pd_train))
# pd_train = pd_train.drop(pd_train[(pd_train['GrLivArea']>5500) 
#  #                               | (pd_train['SalePrice']>500000)].index)
# #print("Dropping outliers resulted in %d instances in the new dataset" % len(pd_train))
# #pd_train = pd_train.drop(pd_train[pd_train['GarageArea']>1100].index)
# #print("Dropping outliers resulted in %d instances in the new dataset" % len(pd_train))

In [35]:
df6.columns

['MSSubClass',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold',
 'SalePrice',
 'MSZoning_index',
 'LotFrontage_index',
 'Street_index',
 'LotShape_index',
 'LandContour_index',
 'Utilities_index',
 'LotConfig_index',
 'LandSlope_index',
 'Neighborhood_index',
 'Condition1_index',
 'Condition2_index',
 'BldgType_index',
 'HouseStyle_index',
 'RoofStyle_index',
 'RoofMatl_index',
 'Exterior1st_index',
 'Exterior2nd_index',
 'MasVnrType_index',
 'MasVnrArea_index',
 'ExterQual_index',
 'ExterCond_index',
 'Foundation_index',
 'BsmtQual_index',
 'BsmtCond_index',
 'BsmtExp

In [36]:
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder(inputCols= ['MSZoning_index',
 'LotFrontage_index',
 'Street_index',
 'LotShape_index',
 'LandContour_index',
 'Utilities_index',
 'LotConfig_index',
 'LandSlope_index',
 'Neighborhood_index',
 'Condition1_index',
 'Condition2_index',
 'BldgType_index',
 'HouseStyle_index',
 'RoofStyle_index',
 'RoofMatl_index',
 'Exterior1st_index',
 'Exterior2nd_index',
 'MasVnrType_index',
 'MasVnrArea_index',
 'ExterQual_index',
 'ExterCond_index',
 'Foundation_index',
 'BsmtQual_index',
 'BsmtCond_index'], 
                         outputCols=[
 'MSZoningVec',
 'LotFrontageVec',
 'StreetVec',
 'LotShapeVec',
 'LandContourVec',
 'UtilitiesVec',
 'LotConfigVec',
 'LandSlopeVec',
 'NeighborhoodVec',
 'Condition1Vec',
 'Condition2Vec',
 'BldgTypeVec',
 'HouseStyleVec',
 'RoofStyleVec',
 'RoofMatlVec',
 'Exterior1stVec',
 'Exterior2ndVec',
 'MasVnrTypeVec',
 'MasVnrAreaVec',
 'ExterQualVec',
 'ExterCondVec',
 'FoundationVec',
 'BsmtQualVec',
 'BsmtCondVec'])
encodedDF = encoder.fit(df6).transform(df6)
encodedDF.select( 'LotFrontage_index',
 'Street_index',
 'LotShape_index',
 'LandContour_index', 
 'LotFrontageVec',
 'StreetVec',
 'LotShapeVec',
 'LandContourVec').show(5, False)

+-----------------+------------+--------------+-----------------+----------------+-------------+-------------+--------------+
|LotFrontage_index|Street_index|LotShape_index|LandContour_index|LotFrontageVec  |StreetVec    |LotShapeVec  |LandContourVec|
+-----------------+------------+--------------+-----------------+----------------+-------------+-------------+--------------+
|6.0              |0.0         |0.0           |0.0              |(111,[6],[1.0]) |(2,[0],[1.0])|(4,[0],[1.0])|(4,[0],[1.0]) |
|3.0              |0.0         |0.0           |0.0              |(111,[3],[1.0]) |(2,[0],[1.0])|(4,[0],[1.0])|(4,[0],[1.0]) |
|13.0             |0.0         |1.0           |0.0              |(111,[13],[1.0])|(2,[0],[1.0])|(4,[1],[1.0])|(4,[0],[1.0]) |
|1.0              |0.0         |1.0           |0.0              |(111,[1],[1.0]) |(2,[0],[1.0])|(4,[1],[1.0])|(4,[0],[1.0]) |
|42.0             |0.0         |1.0           |0.0              |(111,[42],[1.0])|(2,[0],[1.0])|(4,[1],[1.0])|(4,[0],[

In [37]:
encodedDF.columns

['MSSubClass',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold',
 'SalePrice',
 'MSZoning_index',
 'LotFrontage_index',
 'Street_index',
 'LotShape_index',
 'LandContour_index',
 'Utilities_index',
 'LotConfig_index',
 'LandSlope_index',
 'Neighborhood_index',
 'Condition1_index',
 'Condition2_index',
 'BldgType_index',
 'HouseStyle_index',
 'RoofStyle_index',
 'RoofMatl_index',
 'Exterior1st_index',
 'Exterior2nd_index',
 'MasVnrType_index',
 'MasVnrArea_index',
 'ExterQual_index',
 'ExterCond_index',
 'Foundation_index',
 'BsmtQual_index',
 'BsmtCond_index',
 'BsmtExp

In [38]:
encodedDF.show()

+----------+-------+-----------+-----------+---------+------------+----------+----------+---------+-----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+------------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+-------+------+------+---------+--------------+-----------------+------------+--------------+-----------------+---------------+---------------+---------------+------------------+----------------+----------------+--------------+----------------+---------------+--------------+-----------------+-----------------+----------------+----------------+---------------+---------------+----------------+--------------+--------------+------------------+------------------+------------------+-------------+---------------+----------------+----------------+-----------------+----------------+----------------+-----------------+------------------+----------------+-------------

VectorAssembling

In [39]:
# Import VectorAssembler from pyspark.ml.feature package
from pyspark.ml.feature import VectorAssembler
# Create a list of all the variables that you want to create feature vectors
# These features are then further used for training model
features_col = ['MSSubClass',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold',
 'MSZoningVec',
 'LotFrontageVec',
 'StreetVec',
 'LotShapeVec',
 'LandContourVec',
 'UtilitiesVec',
 'LotConfigVec',
 'LandSlopeVec',
 'NeighborhoodVec',
 'Condition1Vec',
 'Condition2Vec',
 'BldgTypeVec',
 'HouseStyleVec',
 'RoofStyleVec',
 'RoofMatlVec',
 'Exterior1stVec',
 'Exterior2ndVec',
 'MasVnrTypeVec',
 'MasVnrAreaVec',
 'ExterQualVec',
 'ExterCondVec',
 'FoundationVec',
 'BsmtQualVec',
 'BsmtCondVec']
# Create the VectorAssembler object
assembler = VectorAssembler(inputCols= features_col, outputCol= "features")
assembledDF = assembler.transform(encodedDF)
assembledDF.select("features").show(5, False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                                                                                                                                                                                                                         |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

VectorIndexing

In [40]:
# Import VectorIndexer from pyspark.ml.feature package
from pyspark.ml.feature import VectorIndexer
# Create a list of all the raw features
# VectorIndexer will automatically identify the categorical columns and index them
featurecol = ['MSSubClass',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold',
 'MSZoning_index',
 'LotFrontage_index',
 'Street_index',
 'LotShape_index',
 'LandContour_index',
 'Utilities_index',
 'LotConfig_index',
 'LandSlope_index',
 'Neighborhood_index',
 'Condition1_index',
 'Condition2_index',
 'BldgType_index',
 'HouseStyle_index',
 'RoofStyle_index',
 'RoofMatl_index',
 'Exterior1st_index',
 'Exterior2nd_index',
 'MasVnrType_index',
 'MasVnrArea_index',
 'ExterQual_index',
 'ExterCond_index',
 'Foundation_index',
 'BsmtQual_index',
 'BsmtCond_index',
 'BsmtExposure_index',
 'BsmtFinType1_index',
 'BsmtFinType2_index',
 'Heating_index',
 'HeatingQC_index',
 'CentralAir_index',
 'Electrical_index',
 'KitchenQual_index',
 'Functional_index',
 'GarageType_index',
 'GarageYrBlt_index',
 'GarageFinish_index',
 'GarageQual_index',
 'GarageCond_index',
 'PavedDrive_index',
 'SaleType_index',
 'SaleCondition_index']

# Create the VectorAssembler object
assembler = VectorAssembler(inputCols= featurecol, outputCol= "features")
assembledDF = assembler.transform(df6)

# Create the VectorIndexer object. It only take feature column
vecindexer = VectorIndexer(inputCol= "features", outputCol= "indexed_features")
# Fit the vectorindexer object on the output of the vectorassembler data and transform
vecindexedDF = vecindexer.fit(assembledDF).transform(assembledDF)
vecindexedDF.select("features", "indexed_features").show(5, False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                                                                                                                                |indexed_features                                              

Scaling The Data(StandardScaler)

In [41]:
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors, VectorUDT

# Define a udf that converts sparse vector into dense vector
# You cannot create your own custom function and run that against the data directly. 
# In Spark, You have to register the function first using udf function
sparseToDense = F.udf(lambda v : Vectors.dense(v), VectorUDT())

# We then call the function here passing the column name on which the function has to be applied
densefeatureDF = assembledDF.withColumn('features_array', sparseToDense('features'))

densefeatureDF.select("features", "features_array").show(5, False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                                                                                                                                |features_array                   

In [42]:
densefeatureDF.printSchema()

root
 |-- MSSubClass: integer (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- BsmtFinSF1: integer (nullable = true)
 |-- BsmtFinSF2: integer (nullable = true)
 |-- BsmtUnfSF: integer (nullable = true)
 |-- TotalBsmtSF: integer (nullable = true)
 |-- 1stFlrSF: integer (nullable = true)
 |-- 2ndFlrSF: integer (nullable = true)
 |-- LowQualFinSF: integer (nullable = true)
 |-- GrLivArea: integer (nullable = true)
 |-- BsmtFullBath: integer (nullable = true)
 |-- BsmtHalfBath: integer (nullable = true)
 |-- FullBath: integer (nullable = true)
 |-- HalfBath: integer (nullable = true)
 |-- BedroomAbvGr: integer (nullable = true)
 |-- KitchenAbvGr: integer (nullable = true)
 |-- TotRmsAbvGrd: integer (nullable = true)
 |-- Fireplaces: integer (nullable = true)
 |-- GarageCars: integer (nullable = true)
 |-

In [43]:
# Import StandardScaler from pyspark.ml.feature package
from pyspark.ml.feature import StandardScaler

# Create the StandardScaler object. It only take feature column (dense vector)
stdscaler = StandardScaler(inputCol= "features_array", outputCol= "scaledfeatures")

# Fit the StandardScaler object on the output of the dense vector data and transform
stdscaledDF = stdscaler.fit(densefeatureDF).transform(densefeatureDF)
stdscaledDF.select("scaledfeatures" ).show(5, False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|scaledfeatures                                                                                                              

In [44]:
# We spilt the data into 70-30 set
# Training Set - 70% obesevations
# Testing Set - 30% observations
trainDF, testDF =  stdscaledDF.randomSplit([0.7,0.3], seed = 2020)

# print the count of observations in each set
print("Observations in training set = ", trainDF.count())
print("Observations in testing set = ", testDF.count())

Observations in training set =  1053
Observations in testing set =  407


In [45]:
# # import the LinearRegression function from the pyspark.ml.classification package
# from pyspark.ml.regression import LinearRegression

# # Build the LinearRegression object 'lr' by setting the required parameters
# #lr = LinearRegression(featuresCol="features", labelCol="SalePrice")
# lr = LinearRegression(featuresCol = 'features', labelCol='SalePrice', maxIter=10, 
#                       regParam=0.8, elasticNetParam=0)

# # fit the LinearRegression object on the training data
# lrmodel = lr.fit(trainDF)

# #This LinearRegressionModel can be used as a transformer to perform prediction on the testing data
# predictonDF = lrmodel.transform(testDF)

# predictonDF.select("SalePrice").show(10,False)

In [46]:

from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol = 'features', labelCol='SalePrice', maxIter=10, 
                      regParam=0.8, elasticNetParam=0.1)
lr_model = lr.fit(trainDF)

trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

lr_predictions = lr_model.transform(testDF)
lr_predictions.select("prediction","SalePrice","features").show(5)

from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="SalePrice",metricName="r2")
print("R Squared (R2) on val data = %g" % lr_evaluator.evaluate(lr_predictions))

RMSE: 33317.632146
r2: 0.823316
+------------------+---------+--------------------+
|        prediction|SalePrice|            features|
+------------------+---------+--------------------+
| 80129.05249372567|   115000|(74,[0,1,2,3,4,5,...|
|  70431.5449722358|   109500|(74,[0,1,2,3,4,5,...|
| 141641.4093154124|   138000|(74,[0,1,2,3,4,5,...|
| 66087.39795058127|    72500|(74,[0,1,2,3,4,5,...|
|205291.46612472436|   193879|(74,[0,1,2,3,4,5,...|
+------------------+---------+--------------------+
only showing top 5 rows

R Squared (R2) on val data = 0.857565


In [47]:
test_df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |--

In [48]:
from pyspark.sql.types import IntegerType
df3_test = test_df
df4_test = df3_test \
  .withColumn("BsmtFinSF1" ,
              df3_test["BsmtFinSF1"]
              .cast(IntegerType()))   \
  .withColumn("BsmtFinSF2",
              df3_test["BsmtFinSF2"]
              .cast(IntegerType()))    \
  .withColumn("BsmtUnfSF" ,
              df3_test["BsmtUnfSF"]
              .cast(IntegerType())) \
  .withColumn("TotalBsmtSF" ,
              df3_test["TotalBsmtSF"]
              .cast(IntegerType())) \
  .withColumn("BsmtFullBath" ,
              df3_test["BsmtFullBath"]
              .cast(IntegerType())) \
  .withColumn("BsmtHalfBath" ,
              df3_test["BsmtHalfBath"]
              .cast(IntegerType())) \
  .withColumn("GarageCars" ,
              df3_test["GarageCars"]
              .cast(IntegerType())) \
  .withColumn("GarageArea" ,
              df3_test["GarageArea"]
              .cast(IntegerType())) 

In [49]:
t_df3 = df4_test.drop('Alley','PoolQC','Fence','MiscFeature','FireplaceQu')

t_df5 = t_df3.drop('Id')

test_string_columns = []

for col, dtype in t_df5.dtypes:
    if dtype == 'string':
        test_string_columns.append(col)


from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

indexers = [StringIndexer(inputCol=column, outputCol=column+'_index', handleInvalid='keep').fit(t_df5) for column in test_string_columns]


pipeline = Pipeline(stages=indexers)
test_indexed = pipeline.fit(t_df5).transform(t_df5)


t_df6= test_indexed.drop(*test_string_columns)
t_df6.show()

+----------+-------+-----------+-----------+---------+------------+----------+----------+---------+-----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+------------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+-------+------+------+--------------+-----------------+------------+--------------+-----------------+---------------+---------------+---------------+------------------+----------------+----------------+--------------+----------------+---------------+--------------+-----------------+-----------------+----------------+----------------+---------------+---------------+----------------+--------------+--------------+------------------+------------------+------------------+-------------+---------------+----------------+----------------+-----------------+----------------+----------------+-----------------+------------------+----------------+----------------+------

In [50]:
t_df6.columns

['MSSubClass',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold',
 'MSZoning_index',
 'LotFrontage_index',
 'Street_index',
 'LotShape_index',
 'LandContour_index',
 'Utilities_index',
 'LotConfig_index',
 'LandSlope_index',
 'Neighborhood_index',
 'Condition1_index',
 'Condition2_index',
 'BldgType_index',
 'HouseStyle_index',
 'RoofStyle_index',
 'RoofMatl_index',
 'Exterior1st_index',
 'Exterior2nd_index',
 'MasVnrType_index',
 'MasVnrArea_index',
 'ExterQual_index',
 'ExterCond_index',
 'Foundation_index',
 'BsmtQual_index',
 'BsmtCond_index',
 'BsmtExposure_index',


In [51]:

from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder(inputCols= ['MSZoning_index',
 'LotFrontage_index',
 'Street_index',
 'LotShape_index',
 'LandContour_index',
 'Utilities_index',
 'LotConfig_index',
 'LandSlope_index',
 'Neighborhood_index',
 'Condition1_index',
 'Condition2_index',
 'BldgType_index',
 'HouseStyle_index',
 'RoofStyle_index',
 'RoofMatl_index',
 'Exterior1st_index',
 'Exterior2nd_index',
 'MasVnrType_index',
 'MasVnrArea_index',
 'ExterQual_index',
 'ExterCond_index',
 'Foundation_index',
 'BsmtQual_index',
 'BsmtCond_index'], 
                         outputCols=[
 'MSZoningVec',
 'LotFrontageVec',
 'StreetVec',
 'LotShapeVec',
 'LandContourVec',
 'UtilitiesVec',
 'LotConfigVec',
 'LandSlopeVec',
 'NeighborhoodVec',
 'Condition1Vec',
 'Condition2Vec',
 'BldgTypeVec',
 'HouseStyleVec',
 'RoofStyleVec',
 'RoofMatlVec',
 'Exterior1stVec',
 'Exterior2ndVec',
 'MasVnrTypeVec',
 'MasVnrAreaVec',
 'ExterQualVec',
 'ExterCondVec',
 'FoundationVec',
 'BsmtQualVec',
 'BsmtCondVec'])
t_encodedDF = encoder.fit(t_df6).transform(t_df6)
t_encodedDF.select( 'LotFrontage_index',
 'Street_index',
 'LotShape_index',
 'LandContour_index', 
 'LotFrontageVec',
 'StreetVec',
 'LotShapeVec',
 'LandContourVec').show(5, False)

 # Import VectorAssembler from pyspark.ml.feature package
from pyspark.ml.feature import VectorAssembler
# Create a list of all the variables that you want to create feature vectors
# These features are then further used for training model
features_col = ['MSSubClass',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold',
 'MSZoningVec',
 'LotFrontageVec',
 'StreetVec',
 'LotShapeVec',
 'LandContourVec',
 'UtilitiesVec',
 'LotConfigVec',
 'LandSlopeVec',
 'NeighborhoodVec',
 'Condition1Vec',
 'Condition2Vec',
 'BldgTypeVec',
 'HouseStyleVec',
 'RoofStyleVec',
 'RoofMatlVec',
 'Exterior1stVec',
 'Exterior2ndVec',
 'MasVnrTypeVec',
 'MasVnrAreaVec',
 'ExterQualVec',
 'ExterCondVec',
 'FoundationVec',
 'BsmtQualVec',
 'BsmtCondVec']
# Create the VectorAssembler object
t_assembler = VectorAssembler(inputCols= features_col, outputCol= "features")
t_assembledDF = t_assembler.transform(t_encodedDF)
t_assembledDF.select("features").show(5, False)

# Import VectorIndexer from pyspark.ml.feature package
from pyspark.ml.feature import VectorIndexer
# Create a list of all the raw features
# VectorIndexer will automatically identify the categorical columns and index them
featurecol = ['MSSubClass',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold',
 'MSZoning_index',
 'LotFrontage_index',
 'Street_index',
 'LotShape_index',
 'LandContour_index',
 'Utilities_index',
 'LotConfig_index',
 'LandSlope_index',
 'Neighborhood_index',
 'Condition1_index',
 'Condition2_index',
 'BldgType_index',
 'HouseStyle_index',
 'RoofStyle_index',
 'RoofMatl_index',
 'Exterior1st_index',
 'Exterior2nd_index',
 'MasVnrType_index',
 'MasVnrArea_index',
 'ExterQual_index',
 'ExterCond_index',
 'Foundation_index',
 'BsmtQual_index',
 'BsmtCond_index',
 'BsmtExposure_index',
 'BsmtFinType1_index',
 'BsmtFinType2_index',
 'Heating_index',
 'HeatingQC_index',
 'CentralAir_index',
 'Electrical_index',
 'KitchenQual_index',
 'Functional_index',
 'GarageType_index',
 'GarageYrBlt_index',
 'GarageFinish_index',
 'GarageQual_index',
 'GarageCond_index',
 'PavedDrive_index',
 'SaleType_index',
 'SaleCondition_index']

# Create the VectorAssembler object
t_assembler = VectorAssembler(inputCols= featurecol, outputCol= "features",handleInvalid='keep')
t_assembledDF = t_assembler.transform(t_df6)

# Create the VectorIndexer object. It only take feature column
t_vecindexer = VectorIndexer(inputCol= "features", outputCol= "indexed_features")
# Fit the vectorindexer object on the output of the vectorassembler data and transform
t_vecindexedDF = t_vecindexer.fit(t_assembledDF).transform(t_assembledDF)
t_vecindexedDF.select("features", "indexed_features").show(5, False)




+-----------------+------------+--------------+-----------------+----------------+-------------+-------------+--------------+
|LotFrontage_index|Street_index|LotShape_index|LandContour_index|LotFrontageVec  |StreetVec    |LotShapeVec  |LandContourVec|
+-----------------+------------+--------------+-----------------+----------------+-------------+-------------+--------------+
|2.0              |0.0         |0.0           |0.0              |(116,[2],[1.0]) |(2,[0],[1.0])|(4,[0],[1.0])|(4,[0],[1.0]) |
|28.0             |0.0         |1.0           |0.0              |(116,[28],[1.0])|(2,[0],[1.0])|(4,[1],[1.0])|(4,[0],[1.0]) |
|13.0             |0.0         |1.0           |0.0              |(116,[13],[1.0])|(2,[0],[1.0])|(4,[1],[1.0])|(4,[0],[1.0]) |
|17.0             |0.0         |1.0           |0.0              |(116,[17],[1.0])|(2,[0],[1.0])|(4,[1],[1.0])|(4,[0],[1.0]) |
|30.0             |0.0         |1.0           |1.0              |(116,[30],[1.0])|(2,[0],[1.0])|(4,[1],[1.0])|(4,[1],[

In [52]:
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors, VectorUDT

# Define a udf that converts sparse vector into dense vector
# You cannot create your own custom function and run that against the data directly. 
# In Spark, You have to register the function first using udf function
t_sparseToDense = F.udf(lambda v : Vectors.dense(v), VectorUDT())

# We then call the function here passing the column name on which the function has to be applied
t_densefeatureDF = t_assembledDF.withColumn('features_array', sparseToDense('features'))

t_densefeatureDF.select("features", "features_array").show(5, False)

# Import StandardScaler from pyspark.ml.feature package
from pyspark.ml.feature import StandardScaler

# Create the StandardScaler object. It only take feature column (dense vector)
t_stdscaler = StandardScaler(inputCol= "features_array", outputCol= "scaledfeatures")

# Fit the StandardScaler object on the output of the dense vector data and transform
t_stdscaledDF = t_stdscaler.fit(t_densefeatureDF).transform(t_densefeatureDF)
t_stdscaledDF.select("scaledfeatures" ).show(5, False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                                                                                                          |features_array                                                                

In [53]:
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors, VectorUDT

# Define a udf that converts sparse vector into dense vector
# You cannot create your own custom function and run that against the data directly. 
# In Spark, You have to register the function first using udf function
t_sparseToDense = F.udf(lambda v : Vectors.dense(v), VectorUDT())

# We then call the function here passing the column name on which the function has to be applied
t_densefeatureDF = t_assembledDF.withColumn('features_array', sparseToDense('features'))

t_densefeatureDF.select("features", "features_array").show(5, False)

# Import StandardScaler from pyspark.ml.feature package
from pyspark.ml.feature import StandardScaler

# Create the StandardScaler object. It only take feature column (dense vector)
t_stdscaler = StandardScaler(inputCol= "features_array", outputCol= "scaledfeatures")

# Fit the StandardScaler object on the output of the dense vector data and transform
t_stdscaledDF = t_stdscaler.fit(t_densefeatureDF).transform(t_densefeatureDF)
t_stdscaledDF.select("scaledfeatures" ).show(5, False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                                                                                                          |features_array                                                                

In [54]:
t_predictonDF = lr_model.transform(t_stdscaledDF)


In [55]:
t_predictonDF.select('Prediction').show(5)

+------------------+
|        Prediction|
+------------------+
| 92709.78232681658|
|171391.33162348135|
|166421.28201107704|
| 194859.7121313028|
| 197891.4214050481|
+------------------+
only showing top 5 rows



In [56]:
p_test = test_df.toPandas()
id_test = p_test['Id']


In [57]:
pd_t_predictonDF = t_predictonDF.toPandas()


In [58]:
pd_t_predictonDF.columns

Index(['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold', 'MSZoning_index', 'LotFrontage_index',
       'Street_index', 'LotShape_index', 'LandContour_index',
       'Utilities_index', 'LotConfig_index', 'LandSlope_index',
       'Neighborhood_index', 'Condition1_index', 'Condition2_index',
       'BldgType_index', 'HouseStyle_index', 'RoofStyle_index',
       'RoofMatl_index', 'Exterior1st_index', 'Exterior2nd_index',
       'MasVnrType_index', 'MasVnrArea_index', 'ExterQual_index',
       'ExterCond_index', 'Foundation_index', 'BsmtQual_index',


In [59]:
import pandas as pd
res = pd.concat([id_test,pd_t_predictonDF['prediction']],axis=1)


In [60]:
pd_t_predictonDF['prediction'].head()

0     92709.782327
1    171391.331623
2    166421.282011
3    194859.712131
4    197891.421405
Name: prediction, dtype: float64

In [61]:
res.head()

,Id,prediction
0,1461,92709.782327
1,1462,171391.331623
2,1463,166421.282011
3,1464,194859.712131
4,1465,197891.421405


In [62]:
res.to_csv('result.csv')

In [63]:
type(id_test)

pandas.core.series.Series

In [64]:
type(test_df)

pyspark.sql.dataframe.DataFrame

In [65]:
lr_model.save('Model.sav')


In [66]:

from pyspark.ml.regression import LinearRegressionModel

lrM = LinearRegressionModel.load('Model.sav')
Prediction2 = lrM.transform(t_stdscaledDF)
Prediction2.show()

+----------+-------+-----------+-----------+---------+------------+----------+----------+---------+-----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+------------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+-------+------+------+--------------+-----------------+------------+--------------+-----------------+---------------+---------------+---------------+------------------+----------------+----------------+--------------+----------------+---------------+--------------+-----------------+-----------------+----------------+----------------+---------------+---------------+----------------+--------------+--------------+------------------+------------------+------------------+-------------+---------------+----------------+----------------+-----------------+----------------+----------------+-----------------+------------------+----------------+----------------+------

In [ ]:
# # import Pipeline from pyspark.ml package
# from pyspark.ml import Pipeline

# # Build the pipeline object by providing stages(transformers + Estimator) 
# # that you need the dataframe to pass through
# # Transfoermers - binarizer, bucketizer, indexers, encoder, assembler
# # Estimator - lr
# lrpipeline = Pipeline(stages=[binarizer, bucketizer, indexers, encoder, assembler, lr])

# # fit the pipeline for the trainind data
# lrpipelinemodel = lrpipeline.fit(trainDF)

# # transform the data
# lrpipelinepredicted = lrpipelinemodel.transform(testDF)

# # view some of the columns generated
# lrpipelinepredicted.select('label', 'rawPrediction', 'probability', 'prediction').show(10)

In [ ]:
# # use save() method to save the model
# # write().overwrite() is usually used when you want to replace the older model with a new one
# # It might happen that you wish to retrain your model and save it at the same the place
# lrpipelinemodel.write().overwrite().save("/FileStore/models/lrmodel")

In [ ]:
# # import PipelineModel from pyspark.ml package
# from pyspark.ml import PipelineModel

# # load the model from the location it is stored
# # The loaded model acts as PipelineModel
# pipemodel = PipelineModel.load("/FileStore/models/lrmodel")

# # use the PipelineModel object to perform prediciton on test data. 
# # Use .transform() to perfrom prediction
# prediction = pipemodel.transform(testDF)

# # print the results
# prediction.select('label', 'rawPrediction', 'probability', 'prediction').show(5)